In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import missingno
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.shape

In [ ]:
df.info()
df.head()

In [ ]:
df['Outcome']= df['Outcome'].astype('category')
df['Outcome'].dtype

In [ ]:
# generate preview of entries with null values
df.isnull().any(axis=None)
    
    

**No missing values**

In [ ]:
sns.heatmap(df.isnull(), cbar=False)

In [ ]:
if len(df[df.duplicated()==True]) > 0:
    print(f'{len(df[df.duplicated()==True])}duplicate values')
else:
    print('No duplicate Values')

In [ ]:
df.describe()

In [ ]:
sns.pairplot(df)

> No certain patterns can be identified between each other

In [ ]:
sns.heatmap(df.corr(),annot=True)

> None of the attributes have high correlation between each other
. Every correlation value is less than 5 

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 10))

# fig.suptitle('Pokemon Stats by Generation')

sns.kdeplot(ax=axes[0, 0],data=df['Pregnancies'])
sns.kdeplot(ax=axes[0, 1],data=df['Glucose'])
sns.kdeplot(ax=axes[0, 2],data=df['BloodPressure'])
sns.kdeplot(ax=axes[0, 3],data=df['SkinThickness'])
sns.kdeplot(ax=axes[1, 0],data=df['Insulin'])
sns.kdeplot(ax=axes[1, 1],data=df['BMI'])
sns.kdeplot(ax=axes[1, 2],data=df['DiabetesPedigreeFunction'])
sns.kdeplot(ax=axes[1, 3],data=df['Age'])



In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(18, 10))
sns.boxplot(ax=axes[0, 0],data=df['Pregnancies'])
sns.boxplot(ax=axes[0, 1],data=df['Glucose'])
sns.boxplot(ax=axes[0, 2],data=df['BloodPressure'])
sns.boxplot(ax=axes[0, 3],data=df['SkinThickness'])
sns.boxplot(ax=axes[1, 0],data=df['Insulin'])
sns.boxplot(ax=axes[1, 1],data=df['BMI'])
sns.boxplot(ax=axes[1, 2],data=df['DiabetesPedigreeFunction'])
sns.boxplot(ax=axes[1, 3],data=df['Age'])


In [ ]:
len(df[df['Pregnancies']>13])
df[df['Pregnancies']>13]


In [ ]:
# df1 = df[df['SkinThickness']>90].drop(axis=1,index=)
print(np.where(df['SkinThickness']>90))
df1=df.drop(index=579)

In [ ]:
df1.iloc[579]

In [ ]:
sns.boxplot(data =df1['SkinThickness'])

In [ ]:
print(np.where(df1['Glucose']==0))
df1.iloc[[75,182, 342, 349, 502]]

*Glucose levels cant be zero *

In [ ]:
df2 = df1.drop(index=[75,182, 342, 349, 502])

In [ ]:
df2.shape

In [ ]:
plt.figure(figsize=(10, 8))
df.Age.hist(bins=df['Age'].nunique())
plt.xlabel('Age')
plt.ylabel('Count')

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

X = df2.drop(['Outcome'], axis=1)
y = df2['Outcome']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# cat_columns = []
num_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI', 'DiabetesPedigreeFunction', 'Age']


ct = make_column_transformer(
    (MinMaxScaler(), num_columns),
    (StandardScaler(), num_columns),
    remainder='passthrough'
)

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)


In [ ]:


from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        
    elif train==False:
        pred = clf.predict(X_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print("Test Result:\n================================================")        
        print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")



In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)

print_score(lr_clf, X_train, y_train, X_test, y_test, train=True)
print_score(lr_clf, X_train, y_train, X_test, y_test, train=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000)
rf_clf.fit(X_train, y_train)

print_score(rf_clf, X_train, y_train, X_test, y_test, train=True)
print_score(rf_clf, X_train, y_train, X_test, y_test, train=False)


**Random Forest classifier overfits**